<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/Tensorflow/Model%E3%82%92%E3%82%BB%E3%83%BC%E3%83%96%E3%81%97%E3%81%A6%E3%83%AA%E3%82%B9%E3%83%88%E3%82%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q h5py pyyaml

In [2]:
from __future__ import absolute_import, division, print_function

import os
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.3.0'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) /255.0

11493376/11490434 [==============================] - 0s 0us/step


## モデルを定義する

In [6]:
def create_model():
    model = tf.keras.models.Sequential([
            keras.layers.Dense(512, activation = 'relu', input_shape=(784,)),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(10, activation = 'softmax')
    ])

    model.compile(optimizer = 'adam',
                  loss = 'sparse_categorical_crossentropy',
                  metrics = ['accuracy'])
    return model
model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 訓練の間にチェックポイントをセーブする
* 主要なユースケースは訓練の間と最後にチェックポイントを自動的にセーブすることです
    * 中止したところから訓練を選択して使用
* tf.keras.callbacks.ModelCheckpointはこのタスクを遂行するcallbackです

In [7]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only = True,
                                                 verbose = 1)
model = create_model()
model.fit(train_images, train_labels, epochs = 10,
          validation_data = (test_images, test_labels),
          callbacks = [cp_callback])

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.2251 - accuracy: 0.6451
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 1.1587 - accuracy: 0.6680 - val_loss: 0.7171 - val_accuracy: 0.7730
Epoch 2/10
27/32 [========================>.....] - ETA: 0s - loss: 0.4307 - accuracy: 0.8808
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.4167 - accuracy: 0.8840 - val_loss: 0.5257 - val_accuracy: 0.8430
Epoch 3/10
27/32 [========================>.....] - ETA: 0s - loss: 0.2946 - accuracy: 0.9259
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.2807 - accuracy: 0.9270 - val_loss: 0.4871 - val_accuracy: 0.8510
Epoch 4/10
26/32 [=======================>......] - ETA: 0s - loss: 0.2148 - accuracy: 0.9555
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 

## 未使用の未訓練のモデルを再構築してみる
* 偶然であっちゃったみたいなだけ
* だって訓練してないもの

In [11]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print('Untrained model, accuracy:{:5.2f}%'.format(100*acc))

32/32 [==============================] - 0s 2ms/step - loss: 2.3830 - accuracy: 0.0960
Untrained model, accuracy: 9.60%


## そっから、チェックポイントから重みをロードして、再評価します

In [13]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 [==============================] - 0s 2ms/step - loss: 0.4264 - accuracy: 0.8710
Restored model, accuracy: 87.10%


## check point callbackオプション 引数とか

In [14]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose = 1,save_weights_only = True,
    # save weights, every 5-epochs.
    period = 5
)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images, test_labels),
          verbose = 0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


## 結果を見てみる...?

In [15]:
import pathlib
# Sort the checkpoints by modification time.
checkpoints = pathlib.Path(checkpoint_dir).glob("*.index")
checkpoints = sorted(checkpoints, key = lambda cp:cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
latest = str(checkpoints[-1])

checkpoints

[PosixPath('training_2/cp-0005.ckpt'),
 PosixPath('training_2/cp-0010.ckpt'),
 PosixPath('training_2/cp-0015.ckpt'),
 PosixPath('training_2/cp-0020.ckpt'),
 PosixPath('training_2/cp-0025.ckpt'),
 PosixPath('training_2/cp-0030.ckpt'),
 PosixPath('training_2/cp-0035.ckpt'),
 PosixPath('training_2/cp-0040.ckpt'),
 PosixPath('training_2/cp-0045.ckpt'),
 PosixPath('training_2/cp-0050.ckpt')]

## loadしよう

In [18]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print(acc)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 [==============================] - 0s 2ms/step - loss: 0.5002 - accuracy: 0.8740
0.8740000128746033
Restored model, accuracy: 87.40%


## 重みを手動でセーブする
* Model.save_weightsメソッドを使用します

In [19]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 [==============================] - 0s 2ms/step - loss: 0.5002 - accuracy: 0.8740
Restored model, accuracy: 87.40%


## モデル全体をセーブする
* モデル全体を
    * 重み値、モデルの構成、optimizerの構成さえもファイルにセーブできる
* 標準HDFSを使った基本的なセーブ・フォーマットを提供

In [20]:
model = create_model()
model.fit(train_images, train_labels, epochs = 5)
# Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.2381 - accuracy: 0.6250
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4497 - accuracy: 0.8880
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.3081 - accuracy: 0.9160
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2222 - accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1613 - accuracy: 0.9640


In [21]:
# Recreate the exact same model, including weights and optimizer
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [23]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy:{:5.2f}%".format(100*acc))

32/32 [==============================] - 0s 2ms/step - loss: 0.4211 - accuracy: 0.0840
Restored model, accuracy: 8.40%
